# **Бот рейтинга**
![alt text](rating.jpg)

**Меню для BotFather:**   
component - сообщить текущий рейтинг

In [26]:
# PersonalRatingBot @PersonalRatingBot
import pandas as pd
import numpy as np
import os
import sqlite3
import telebot
from telebot import types

# from requests.exceptions import ConnectionError

# import time
import datetime
from datetime import datetime

# Токен
TOKEN = "7350611722:AAGP6sVlNpkBfxS1p-Vv4pHTYscOPPdgGW8"  # token PersonalRatingBot

messages = {}
# user_status = {}
password_flag = {}
db_path = os.getcwd().replace("\\", "/") + "/db_order.sqlite"

# Создание экземпляра бота
bot = telebot.TeleBot(TOKEN)


# меню (меню)
@bot.message_handler(commands=["component"])
def start(message):
    """меню бота"""
    global act, messages, elite_users
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    query_test = "SELECT * FROM user_status us WHERE us.user_id = ?"
    if len(cur.execute(query_test, (user_id,)).fetchall()) == 0:
        query_insert = "INSERT INTO user_status (user_id, admin_id, date_time, flag) VALUES (?, ?, ?, ?);"
        cur.execute(
            query_insert,
            (
                user_id,
                user_id,
                user_datatime,
                0,
            ),
        )
        conn.commit()

    query_m = """SELECT us.user_id FROM user_status us WHERE us.flag = 1;"""
    df_elite_users = pd.DataFrame(cur.execute(query_m).fetchall(), columns=["user_id"])
    try:
        elite_users = df_elite_users["user_id"].tolist()
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Локальный трабл, ибо {e}""")

    if user_id in elite_users:
        try:
            bot.send_message(
                message.chat.id,
                text=f"""
            {message.from_user.first_name}, в личные сообщения направлен ваш текущий рейтинг по состоянию на {user_datatime}""",
                parse_mode="Markdown",
            )
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Глобальный трабл 1, ибо {e}""")
        try:
            bot.send_message(
                message.from_user.id, f"Ваш рейтинг {message.from_user.id}"
            )
        except Exception as e:
            bot.send_message(message.chat.id, f"""🚫Локальный трабл, ибо {e}""")
    else:
        bot.send_message(
            message.chat.id,
            f"""{message.from_user.first_name}, Вы не сотрудник КЦ Минздрава или не член рабочей группы региона - эта опция для Вас не доступна""",
        )


# обработка сообщений (не меню)
@bot.message_handler(content_types=["text"])
def handle_message(message):
    """обработка сообщений"""
    global act, messages, elite_users
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    user_id = message.from_user.id
    messages[user_id] = message.text
    user_datatime = datetime.fromtimestamp(message.date).strftime("%Y-%m-%d %H:%M:%S")
    try:
        if messages.get(user_id) == "000":
            bot.delete_message(message.chat.id, message.message_id)
            password_flag[user_id] = True
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
            btn1 = types.KeyboardButton("👻Добавить Избранного")
            btn2 = types.KeyboardButton("☠️Прогнать Избранного")
            markup.add(btn1, btn2)
            bot.send_message(
                message.chat.id,
                f"Что Вы хотите сделать, босс?",
                reply_markup=markup,
            )
        elif messages.get(user_id) == "👻Добавить Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, f"""💥Отправьте id нового Избранного""")
            act = 1
        elif messages.get(user_id) == "☠️Прогнать Избранного" and password_flag.get(
            user_id
        ):
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, f"""⚰️Отправьте id обречённого""")
            act = 0
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act == 1
        ):
            query_kill = "DELETE FROM user_status WHERE user_id = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_id, admin_id, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.id, user_datatime, 1),
            )
            conn.commit()
            act = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
        elif (
            messages.get(user_id)
            not in ["👻Добавить Избранного", "☠️Прогнать Избранного"]
            and act == 0
        ):
            query_kill = "DELETE FROM user_status WHERE user_id = ?;"
            cur.execute(query_kill, (messages.get(user_id),))
            conn.commit()
            query_insert = "INSERT INTO user_status (user_id, admin_id, date_time, flag) VALUES (?, ?, ?, ?);"
            cur.execute(
                query_insert,
                (messages.get(user_id), message.from_user.id, user_datatime, 0),
            )
            conn.commit()
            act = 3
            bot.delete_message(message.chat.id, message.message_id - 1)
            bot.send_message(message.chat.id, "👍")
    except Exception as e:
        bot.send_message(message.chat.id, f"""🚫Глобальный трабл 2, ибо {e}""")


# запуск бота
bot.infinity_polling()

2024-08-20 17:36:22,926 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2024-08-20 17:36:22,926 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
